# Intro to Modeling Notebook
Explain thought process and expectations, and why I chose the models I did

In [1]:
# standard python packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# sklearn modeling packages
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# optimize viewing
pd.set_option('max_colwidth', 150)
pd.set_option('display.max_columns', 200)

# NLP Feature Engineering 
or importing preprocessed data EDA ntbk

In [2]:
df = pd.read_pickle('./data/modeling_df.pkl')

In [3]:
df.head()

,date,tweet,username,link,nlikes,nreplies,nretweets,vader_sentiment,clean_tweets
0,2020-02-28,"For everyone comparing COVID-19 to the flu, just a reminder that even with vaccines the flu kills tens of thousands a year. Now imagine a more dea...",MonaIbrahim,https://twitter.com/MonaIbrahim/status/1233542386669641728,3,0,0,negative,everyone compare flu reminder even flu kill ten thousand year imagine much deadly assurance develope affordable hello e pandemic
1,2020-02-28,"“Perhaps more than any other type of medicine, the success of a #vaccine depends on the cooperation of everyone. To protect those who can’t have a...",Moratel_Entles,https://twitter.com/Moratel_Entles/status/1233542196193824769,1,0,0,positive,perhaps much type medicine success depend cooperation everyone protect young promise need massive buy population large
2,2020-02-28,Provide pharmaceutical companies with the opportunity to develop this vaccine and drug as soon as possible and to reach mass production. As you kn...,help_4_Iranian,https://twitter.com/help_4_Iranian/status/1233542052161507328,0,0,0,positive,provide pharmaceutical company opportunity develop drug soon possible reach mass production know drug produce
3,2020-02-28,From John's Hopkins: 1) Immediate risk of COVID-19 remains low in the U.S. The best way to protect yourself is by practicing the same precaution...,TruthFreedom17,https://twitter.com/TruthFreedom17/status/1233541922108694528,1,1,0,positive,john hopkins immediate risk remain low us well way protect practice precaution would use prevent spread respiratory disease currently prevent
4,2020-02-28,If the #stockmarkets keep falling - how confident will you be about a #vaccine for #COVID-19 and funds being available for their development?,notarywise,https://twitter.com/notarywise/status/1233541049903177728,0,0,0,positive,stockmarkets keep fall confident fund available development


# Topic Modeling & Feature Engineering
perhaps this belongs in the above section?

In [4]:
clean_tweets = df['clean_tweets']

In [5]:
tfidf = TfidfVectorizer(min_df=30)

data_tfidf = tfidf.fit_transform(clean_tweets)

In [ ]:
# make sure all columns are suitable for modeling
df_tfidf.dtypes.unique()

In [7]:
type(data_tfidf)

scipy.sparse.csr.csr_matrix

In [13]:
modeling_df = pd.DataFrame(data_tfidf.todense(), columns = tfidf.get_feature_names())

In [15]:
modeling_df.shape

(77259, 3536)

In [16]:
# set X features and y target
X = modeling_df
y = df['vader_sentiment']

In [17]:
X.shape, y.shape

((77259, 3536), (77259,))

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=88)

# Naive Bayes

In [20]:
# instantiate multinomial naive bayes object
nb_clf = MultinomialNB()

# train naive bayes
nb_clf = nb_clf.fit(X_train, y_train)

# predict on train and test
y_train_preds = nb_clf.predict(X_train)
y_test_preds = nb_clf.predict(X_test)

In [21]:
print(accuracy_score(y_train, y_train_preds))
print(accuracy_score(y_test, y_test_preds))

print(f1_score(y_train, y_train_preds))
print(f1_score(y_test, y_test_preds))

0.7252692254590639
0.7115195443955475


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

# Decision Tree

# SVM